### Project Week 06
名企BI班 week05 谢雅楠 20201003

### Thinking
1. 什么是近似最近邻查找，常用的方法有哪些？  
近似最近邻检索，是在牺牲可接受范围内的精度的情况下的最近邻查找，因为是线性复杂度，相比KNN可以提高检索效率，用于大规模数据。  
常用方法有LSH(locality-sensitive hashing)，MinHashLSH、SimHash
  
2. 为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度?  
设集合S1和S2，那么这两列所在的行有下面3种类型：
S1和S2的值都为1，记为X； 只有一个值为1，另一个值为0，记为Y；S1和S2的值都为0，记为Z。  
S1和S2交集的元素个数为x，并集的元素个数为x+y，  
所以sim(S1,S2) = Jaccard(S1,S2) = x/(x+y)。  
h(S1)=h(S2)的概率为从上往下扫描，在碰到Y行之前碰到X行的概率，x/(x+y)。  
所以两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度。  
ref. https://www.cnblogs.com/sddai/p/6110704.html
  
3. SimHash在计算文档相似度的作用是怎样的？  
SimHash得到每篇文档的指纹，通过计算每两篇文档之间的Hamming距离，如果在3以内，可以认为文档相似度较高。

4. 为什么YouTube采用期望观看时间作为评估指标？  
常用的CTR指标对于视频搜索具有一定的欺骗性，点击了不一定观看，观看时长反应喜爱程度，所以作者提出采用期望观看时间作为评估指标。
ref.https://blog.csdn.net/u012102306/article/details/79889609

### Action 1
使用MinHashLSHForest对微博新闻句子进行检索 weibo.txt  
针对某句话进行Query，查找Top-3相似的句子

In [3]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [4]:
# read in file
f = open('./weibos.txt', 'r', encoding='utf-8')
text = f.read()
# split
sentences = re.split('[。！？]', text.replace('\n', ''))  ### \n end mark
if sentences[len(sentences)-1] == '':
    sentences.pop()

In [6]:
# stop words
stop = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]

# get item string
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    # delete stop words
    for i in list(item):
        if i.word not in list(stop):  
            item_str += i.word 
            item_str += " "
    return item_str

In [8]:
# create document list
documents = []
for item_text in sentences:
    item_str = get_item_str(item_text)
    documents.append(item_str)

In [7]:
# create minHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [9]:
# create LSH Forest
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# create index for keys for query
forest.index()

In [14]:
if __name__ == "__main__":
    query_statement = '里皮在国足输给叙利亚之后决定辞职'
    # 1. get strings of query statemtent
    item_str = get_item_str(query_statement)
    # 2. create minHash
    minhash_query = get_minhash(item_str)
    # 3. query in forest
    result = forest.query(minhash_query, 3)
    for i in range(len(result)):
        print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))

1 0.75 ​国足输给叙利亚之后，里皮辞职
4 0.2265625 ​据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决
37 0.484375 国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职
